In [6]:
import requests
import gradio as gr


In [ ]:
# The following is not working as the uploaded file is empty using Gradio in this way
import tempfile
def submitJob_fake(host, port, file_upload, file_format, to_lan):
    url = f"http://{host}:{port}/jobs"
    with open("hello.pdf", 'wb') as f:
        f.write(file_upload.read())
    '''
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
        temp_file.write(file_upload.read())
        temp_path = temp_file.name
    files = {'file': open(temp_path, 'rb')}
    '''
    files = {'file': open("hello.pdf",'rb')}
    data = {'file_format':file_format, 'to_lan':to_lan}
    response = requests.post(url, files= files, data=data)
    return response.text

iface = gr.Interface(fn=submitJob_fake, inputs=["text", "text", "file", "text","text"], outputs="text")
iface.launch()
    

In [7]:
def submitJob(host, port, file_bytes, file_format, to_lan, style):
    url = f"http://{host}:{port}/jobs"
    with open("original.pdf", 'wb') as f:
        f.write(file_bytes)
    files = {'file': open("original.pdf",'rb')}
    data = {'file_format':file_format, 'to_lan':to_lan, 'style':style}
    response = requests.post(url, files= files, data=data)
    return response.text


In [8]:
def queryJob(host, port, job_id):
    url = f"http://{host}:{port}/jobs/{job_id}"
    response = requests.get(url)
    return response.text

In [9]:
import json
def download(host, port, job_id):
    url = f"http://{host}:{port}/jobs/{job_id}"
    response = requests.get(url)
    if response.ok:
        status_dict = json.loads(response.text)
        if status_dict['status'] == "Finished":
            url = f"http://{host}:{port}/translations/{job_id}"
            return f'<a href="{url}" target="_blank">Click here to view the Translated file</a>'
        else:
            return f'job is not finished'
    else:
        return f'job is not found'
    

In [10]:
with gr.Blocks() as submit_job:
    with gr.Row():
        host = gr.Textbox(label = "host", value="localhost")
        port = gr.Textbox(label = "port", value="5003")
    with gr.Row():
        file_format = gr.Dropdown(label = "file format", choices = ["pdf","md"], value="pdf")
        to_lan = gr.Dropdown(label = "target language", value="Chinese")
        style = gr.Textbox(label = "style", value = "plain")
    with gr.Row():
        file_upload = gr.File(label = "file to translate", file_types=["pdf"], type="binary")
    submit_out = gr.Textbox(label = "submit result")
    submit_btn = gr.Button("Submit")
    submit_btn.click(fn=submitJob, inputs=[host, port, file_upload, file_format, to_lan, style], outputs=submit_out)
with gr.Blocks() as query_job:
    with gr.Row():
        job_id = gr.Textbox(label = "job id", placeholder="The job id returned when you submit the job")
        query_output = gr.Textbox(label = "job status")
    query_btn = gr.Button("Query status of a particular job")
    download_btn = gr.Button("Download the translated file for a particular job (make sure the status is Finished)")
    query_btn.click(fn=queryJob, inputs=[host, port,job_id], outputs=query_output)
    download_btn.click(fn=download, inputs=[host, port,job_id], outputs=gr.outputs.HTML())
    
demo = gr.TabbedInterface([submit_job, query_job], ["submit job", "query_job"])
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
